In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.appName("Local demo")\
    .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven")\
    .config("spark.jars.packages", "com.microsoft.ml.spark:mmlspark_2.11:1.0.0-rc2-85-fb47ab7f-SNAPSHOT")\
    .getOrCreate()

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

from mmlspark.vw import VowpalWabbitContextualBandit, VowpalWabbitFeaturizer, VectorZipper

from pyspark.ml.tuning import ParamGridBuilder
from pyspark.sql.types import IntegerType
from pyspark.ml import Pipeline

In [3]:
def get_data():
    pandas_df = pd.read_csv('cbdata.csv').rename(columns={"cost": "label"})
    # The dataframe is coalesced into 1 partition because for CFE we want 
    # to learn on the whole dataset serially rather than distributed.
    df = spark.createDataFrame(pandas_df).coalesce(1)
    df = df.withColumn("chosenAction", df["chosenAction"].cast(IntegerType()))
    shared_featurizer = VowpalWabbitFeaturizer(inputCols=[
        'sharedID', "sharedMajor", "sharedHobby", "sharedFavCharacter"]
                                               , outputCol='shared')
    action_featurizers = [
        VowpalWabbitFeaturizer(inputCols=['action1Topic'],
                               outputCol='action1Features'),
        VowpalWabbitFeaturizer(inputCols=['action2Topic'],
                               outputCol='action2Features'),
        VowpalWabbitFeaturizer(inputCols=['action3Topic'],
                               outputCol='action3Features'),
        VowpalWabbitFeaturizer(inputCols=['action4Topic'],
                               outputCol='action4Features'),
        VowpalWabbitFeaturizer(inputCols=['action5Topic'],
                               outputCol='action5Features')
    ]

    action_merger = VectorZipper(inputCols=[
        'action1Features', 'action2Features', 'action3Features',
        'action4Features', 'action5Features'
    ],
                                 outputCol='features')

    pipeline = Pipeline(stages=[
        shared_featurizer, action_featurizers[0], *action_featurizers, action_merger
    ])
    tranformation_pipeline = pipeline.fit(df)
    return tranformation_pipeline.transform(df)


In [15]:
featurized_data = get_data()
estimator1 = VowpalWabbitContextualBandit() \
    .setEpsilon(0.2)\
    .setParallelism(12) \
    .setUseBarrierExecutionMode(False)

paramGrid = []

paramGrid.extend(ParamGridBuilder() \
    .addGrid(estimator1.interactions, [[], ["::"]])\
    .addGrid(estimator1.learningRate, [0.1, 0.05, 0.005]) \
    .addGrid(estimator1.powerT, [0.0, 0.1]) \
    .build())

models = estimator1.parallelFit(featurized_data, paramGrid)
result = pd.concat([model.getPerformanceStatistics().toPandas() for model in models])

In [16]:
result[["arguments", "learningRate", "powerT", "ipsEstimate", "snipsEstimate"]].sort_values(by=['ipsEstimate'])g

,arguments,learningRate,powerT,ipsEstimate,snipsEstimate
0,--cb_adf --cb_explore_adf --cb_type mtr --csoaa_ldf multiline --csoaa_rank --epsilon 0.200000002980232 --learning_rate 0.0500000007450581 --no_stdin --power_t 0.100000001490116 --quadratic ::,0.050,0.1,-0.240092,-0.215007
0,--cb_adf --cb_explore_adf --cb_type mtr --csoaa_ldf multiline --csoaa_rank --epsilon 0.200000002980232 --learning_rate 0.0500000007450581 --no_stdin --power_t 0 --quadratic ::,0.050,0.0,-0.238168,-0.215661
0,--cb_adf --cb_explore_adf --cb_type mtr --csoaa_ldf multiline --csoaa_rank --epsilon 0.200000002980232 --learning_rate 0.00499999988824129 --no_stdin --power_t 0.100000001490116 --quadratic ::,0.005,0.1,-0.219321,-0.199359
0,--cb_adf --cb_explore_adf --cb_type mtr --csoaa_ldf multiline --csoaa_rank --epsilon 0.200000002980232 --learning_rate 0.00499999988824129 --no_stdin --power_t 0 --quadratic ::,0.005,0.0,-0.216244,-0.196082
0,--cb_adf --cb_explore_adf --cb_type mtr --csoaa_ldf multiline --csoaa_rank --epsilon 0.200000002980232 --learning_rate 0.100000001490116 --no_stdin --power_t 0.100000001490116 --quadratic ::,0.100,0.1,-0.215090,-0.196199
0,--cb_adf --cb_explore_adf --cb_type mtr --csoaa_ldf multiline --csoaa_rank --epsilon 0.200000002980232 --learning_rate 0.100000001490116 --no_stdin --power_t 0 --quadratic ::,0.100,0.0,-0.213551,-0.192967
0,--cb_adf --cb_explore_adf --cb_type mtr --csoaa_ldf multiline --csoaa_rank --epsilon 0.200000002980232 --learning_rate 0.100000001490116 --no_stdin --power_t 0,0.100,0.0,-0.188165,-0.159028
0,--cb_adf --cb_explore_adf --cb_type mtr --csoaa_ldf multiline --csoaa_rank --epsilon 0.200000002980232 --learning_rate 0.100000001490116 --no_stdin --power_t 0.100000001490116,0.100,0.1,-0.187395,-0.161904
0,--cb_adf --cb_explore_adf --cb_type mtr --csoaa_ldf multiline --csoaa_rank --epsilon 0.200000002980232 --learning_rate 0.0500000007450581 --no_stdin --power_t 0,0.050,0.0,-0.178164,-0.149652
0,--cb_adf --cb_explore_adf --cb_type mtr --csoaa_ldf multiline --csoaa_rank --epsilon 0.200000002980232 --learning_rate 0.00499999988824129 --no_stdin --power_t 0,0.005,0.0,-0.177779,-0.166248
